In [1]:
# import tensorflow as tf
import numpy as np
import json
import cv2
import os
from deepface import DeepFace
import matplotlib.pyplot as plt
import time

In [2]:
# Folder containing multiple images of the same face
image_folder = 'input_images'

# An output file to store the extracted embeddings
output_file = 'embeddings/embeddings.json'



# A user name of the input images
name = 'Mussa'

# Create a list to store embeddings
all_embeddings = []

# List all image files in the folder
image_files = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.jpg')]


In [3]:
# Extract faces
def extract_faces(img_path):
    print(f"Extracting faces for image: {img_path}")
    imgs = DeepFace.extract_faces(img_path=img_path, enforce_detection=False)
    return imgs

# Create embeddings
def create_embeddings(img_path):
    # print(f"Creating embeddings for image: {img_path}")
    print(f"Creating embeddings...")
    embedding_objs = DeepFace.represent(img_path=img_path, model_name="Facenet", enforce_detection=False)
    return embedding_objs

In [4]:
if len(image_files) > 0:
    for image_path in image_files:
        imgs = extract_faces(image_path)
        if len(imgs) > 0:
            print(f"Found {str(len(imgs))} face{'' if len(imgs) == 1 else 's'} for image: {image_path}")
            
            for i, img in enumerate(imgs):
                if img["confidence"] > 0:
                    print(f' - Face no: {i + 1}, confidence: {img["confidence"]}')
                    # plt.title(f'Face no: {i + 1}, confidence: {img["confidence"]}')
                    # plt.imshow(img["face"])
                    # plt.show()

                    embeddings = create_embeddings(img["face"])
                    if len(embeddings) > 0:
                        for embedding in embeddings:
                            all_embeddings.append(embedding["embedding"])

        else:
            print(f"No faces found for image: {image_path}")

else:
    print("No image files found")

No image files found


In [5]:
# all_embeddings

In [6]:
# Calculate the average embedding (if needed)
if len(all_embeddings) > 0:

    # # Calculate the average embedding
    # average_embedding = np.mean(all_embeddings, axis=0)
    
    # # Store the average embedding in a JSON file
    # data = {
    #     'name': user_name,
    #     'embedding': average_embedding.tolist()
    # }
    existing_data = []

    # Load the existing data from the JSON file
    with open(output_file, 'r') as json_file:
        existing_data = json.load(json_file)

    # Check if the name already exists in the data
    word_exists = False
    for entry in existing_data:
        if entry["name"] == name:
            # Get current time in milliseconds (since epoch)
            current_time_ms = str(int(time.time() * 1000))
            entry["embeddings"][current_time_ms] = all_embeddings
            word_exists = True
            break
    
    # If the word doesn't exist, add a new entry
    if not word_exists:
        # Get current time in milliseconds (since epoch)
        current_time_ms = str(int(time.time() * 1000))
        new_entry = {
            "name": name,
            "embeddings": {
                current_time_ms: all_embeddings
            }
        }
        existing_data.append(new_entry)

    # Serialize the updated data to JSON and save it back to the file
    with open(output_file, 'w') as json_file:
        json.dump(existing_data, json_file, indent=2)
        
    print(f'Done creating embeddings for user: {name}')

else: 
    print("No embeddings generated for the input images")

No embeddings generated for the input images
